**Lista B) Implementação Computacional:**

Usando linguagens de programação (Python, ou Matalb ou Octave, etc) implementar exemplos de modelos considerando:

1. Treliças estáticas

Descrição da Implementação — Treliça Estática
1. Definição da geometria e conectividade
* Foram definidos 3 nós formando um triângulo equilátero.
* A conectividade entre os nós representa 3 barras ligando os pares (1–2), (2–3) e (3–1).

2. Propriedades dos elementos
* Módulo de elasticidade E = 210x10^9 Pa
* Área da seção transversal A = 0,01m^2

3. Montagem da matriz de rigidez global
* Para cada barra, foi calculada a matriz de rigidez local usando coordenadas dos nós.
* As matrizes locais foram transformadas e somadas na matriz de rigidez global
K, considerando os graus de liberdade de cada nó.

4. Aplicação das condições de contorno
* O nó 1 foi fixado (graus de liberdade 0 e 1).
* Os demais graus de liberdade foram considerados livres para deslocamento.

5. Aplicação de carregamento
* Uma força vertical de –1000 N foi aplicada no eixo y do nó 3.

6. Resolução do sistema
* O sistema foi reduzido para considerar apenas os graus de liberdade livres.
* Foi resolvido usando álgebra linear para obter os deslocamentos nodais.

7. Resultados
* O script imprime os deslocamentos U_x e U_y de cada nó, com precisão científica.

In [ ]:
import numpy as np

# Dados dos nós
nodes = np.array([
    [0.0, 0.0],        # Nó 1
    [1.0, 0.0],        # Nó 2
    [0.5, 0.866]       # Nó 3
])

# Conectividade dos elementos (nós iniciais e finais)
elements = [
    (0, 1),  # Barra entre Nó 1 e 2
    (1, 2),  # Barra entre Nó 2 e 3
    (2, 0)   # Barra entre Nó 3 e 1
]

# Propriedades
E = 210e9  # Pa
A = 0.01   # m²

# Número de graus de liberdade
ndof = 2 * len(nodes)
K_global = np.zeros((ndof, ndof))

# Função para matriz de rigidez local
def k_local(E, A, xi, xj):
    L = np.linalg.norm(xj - xi)
    c = (xj[0] - xi[0]) / L
    s = (xj[1] - xi[1]) / L
    k = (E * A / L) * np.array([
        [ c*c,  c*s, -c*c, -c*s],
        [ c*s,  s*s, -c*s, -s*s],
        [-c*c, -c*s,  c*c,  c*s],
        [-c*s, -s*s,  c*s,  s*s]
    ])
    return k

# Montagem da matriz global
for e in elements:
    i, j = e
    xi, xj = nodes[i], nodes[j]
    k_e = k_local(E, A, xi, xj)

    dofs = [2*i, 2*i+1, 2*j, 2*j+1]
    for m in range(4):
        for n in range(4):
            K_global[dofs[m], dofs[n]] += k_e[m, n]

# Aplicar condições de contorno (fixar Nó 1)
fixed_dofs = [0, 1]
free_dofs = list(set(range(ndof)) - set(fixed_dofs))

# Vetor de forças (força vertical em Nó 2)
F = np.zeros(ndof)
F[5] = -1000  # N aplicada no eixo y do Nó 3

# Resolver sistema reduzido
K_reduced = K_global[np.ix_(free_dofs, free_dofs)]
F_reduced = F[free_dofs]
U_reduced = np.linalg.solve(K_reduced, F_reduced)

# Reconstruir vetor de deslocamentos completo
U = np.zeros(ndof)
U[free_dofs] = U_reduced

print("Deslocamentos nos nós:")
for i in range(len(nodes)):
    print(f"Nó {i+1}: Ux = {U[2*i]:.6e} m, Uy = {U[2*i+1]:.6e} m")

Deslocamentos nos nós:
Nó 1: Ux = 0.000000e+00 m, Uy = 0.000000e+00 m
Nó 2: Ux = 0.000000e+00 m, Uy = -3.228871e+09 m
Nó 3: Ux = 2.796203e+09 m, Uy = -1.614436e+09 m
